<a href="https://colab.research.google.com/github/ruben216/realidadepublica/blob/main/Report_RP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Enviando requisições
Será necessário possuir uma chave gerada no portal de transparência. Detalhes no Readme do repositório.

In [27]:
import pandas as pd
import requests
import json
import numpy as np
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
# Chave gerada no portal
chave_portal = '6f98dcedb14a7c1053bc2bd8fb56ba5f'

# É necessário enviar a chave no Header da requisição
header_chave = {'chave-api-dados':chave_portal}

#Variavel que indica se tivemos conteúdo retornado do Portal
conteudo = 1

#Variável que controla a página que estamos iterando
pagina = 1

#Ano que iremos consultar
ano = 2021

#Objeto que irá armazenar os dados retornados de todas as páginas
dados_completos = []

while(conteudo > 0):
  url = f'http://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano={ano}&pagina={pagina}'
  dados_pagina = requests.get(url, verify=True,headers=header_chave).json()
  conteudo = len(dados_pagina)
  if(conteudo > 0):
    dados_completos = dados_completos + dados_pagina
  pagina += 1
  print(pagina)

df_dados_completos = pd.DataFrame(dados_completos)
df_dados_completos.head()


In [33]:
df_dados_completos.to_csv('/drive/MyDrive/Colab Notebooks/df_dados_completos.csv')